# Neuron set examples

In [1]:
import os

import obi_one as obi

### __Initialization:__ Loading a circuit

In [2]:
circuit_path_prefix = "/Users/james/Documents/obi/additional_data/O1_data/"
circ_path = circuit_path_prefix + "O1_data/circuit_config.json"
circuit = obi.Circuit(name="SSCX_O1", path=circ_path)
print(f"Circuit '{circuit}' with {circuit.sonata_circuit.nodes.size} neurons and {circuit.sonata_circuit.edges.size} synapses")
print(f"Default node population: '{circuit.default_population_name}'")

Circuit 'SSCX_O1' with 785632 neurons and 560631825 synapses
Default node population: 'S1nonbarrel_neurons'


### __Example 1:__ Adding node set dict to an existing SONATA circuit object + writing new node set .json file

In [3]:
# Get SONATA circuit object
c = circuit.sonata_circuit
print("..." + str(c.node_sets.content)[-25:])

# Adding a node set to the circuit
obi.NeuronSet.add_node_set_to_circuit(c, {"Layer23": {"layer": [2, 3]}})
print("..." + str(c.node_sets.content)[-55:])

# Adding a node set with an exising name => NOT POSSIBLE
# obi.NeuronSet.add_node_set_to_circuit(c, {"Layer23": {"layer": [2, 3]}})  # AssertionError: Node set 'Layer23' already exists!

# Update/overwrite an existing node set
obi.NeuronSet.add_node_set_to_circuit(c, {"Layer23": ["Layer2", "Layer3"]}, overwrite_if_exists=True)  # Update/overwrite
print("..." + str(c.node_sets.content)[-58:])

# Adding multiple node sets
obi.NeuronSet.add_node_set_to_circuit(c, {"Layer45": ["Layer4", "Layer5"], "Layer56": ["Layer5", "Layer6"]})
print("..." + str(c.node_sets.content)[-124:])

# Add node set from NeuronSet object, resolved in circuit's default node population
neuron_set = obi.CombinedNeuronSet(node_sets=("Layer1", "Layer2", "Layer3"))
obi.NeuronSet.add_node_set_to_circuit(c, {"Layer123": neuron_set.get_node_set_definition(circuit, circuit.default_population_name)})
print("..." + str(c.node_sets.content)[-168:])

# Adding a node sets based on previously added node sets
obi.NeuronSet.add_node_set_to_circuit(c, {"AllLayers": ["Layer123", "Layer4", "Layer56"]})
print("..." + str(c.node_sets.content)[-216:])

# Write new circuit's node set file
obi.NeuronSet.write_circuit_node_set_file(c, output_path="./", file_name="new_node_sets.json", overwrite_if_exists=True)

...211709], 'layer': ['6']}}
...211709], 'layer': ['6']}, 'Layer23': {'layer': [2, 3]}}
...211709], 'layer': ['6']}, 'Layer23': ['Layer2', 'Layer3']}
...211709], 'layer': ['6']}, 'Layer23': ['Layer2', 'Layer3'], 'Layer45': ['Layer4', 'Layer5'], 'Layer56': ['Layer5', 'Layer6']}
...211709], 'layer': ['6']}, 'Layer23': ['Layer2', 'Layer3'], 'Layer45': ['Layer4', 'Layer5'], 'Layer56': ['Layer5', 'Layer6'], 'Layer123': ['Layer1', 'Layer2', 'Layer3']}
...211709], 'layer': ['6']}, 'Layer23': ['Layer2', 'Layer3'], 'Layer45': ['Layer4', 'Layer5'], 'Layer56': ['Layer5', 'Layer6'], 'Layer123': ['Layer1', 'Layer2', 'Layer3'], 'AllLayers': ['Layer123', 'Layer4', 'Layer56']}


### __Example 2:__ Use of different NeuronSet types

<u>Important</u>: In general, the validity of neuron set definitions is not checked during initialization, but when resolved within a specific circuit's node population

#### (a) __ExistingNeuronSet__, wrapper for an existing node set
<u>Note:</u> This neuron set does not resolve to a dict, since the underlying node set is already existing by definition

In [ ]:
neuron_set = obi.PredefinedNeuronSet(node_set="Layer1", sample_percentage=100)
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

PredefinedNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'SSCX_O1':
> Neuron IDs (2695): [    0     1     2 ... 35663 35664 35665]
> Node set dict: ['Layer1']


#### (b) __ExistingNeuronSet__, with random sub-sampling
<u>Note</u>: `sample_percentage` can be an absolute number or fraction

<u>Note 2</u>: Random sub-sampling will enforce resolving into a new node set

In [5]:
neuron_set = obi.PredefinedNeuronSet(node_set="Layer1", sample_percentage=10, sample_seed=1)
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

PredefinedNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'SSCX_O1':
> Neuron IDs (10): [   41   193   222   229 33940 34013 34550 34799 35275 35626]
> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [41, 193, 222, 229, 33940, 34013, 34550, 34799, 35275, 35626]}


#### (c) __CombinedNeuronSet__, based on combining existing (named) node sets

In [ ]:
neuron_set = obi.CombinedNeuronSet(circuit=circuit, population="All", node_sets=("Layer1", "Layer2", "Layer3"), sample_percentage=100)
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

CombinedNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'SSCX_O1':
> Neuron IDs (63530): [     0      1      2 ... 170851 170852 170853]
> Node set dict: ['Layer1', 'Layer2', 'Layer3']


#### (d) __CombinedNeuronSet__, based on combining existing (named) node sets, with random sub-sampling
<u>Note</u>: `sample_percentage` can be an absolute number or fraction

In [7]:
neuron_set = obi.CombinedNeuronSet(node_sets=("Layer1", "Layer2", "Layer3"), sample_percentage=10, sample_seed=0)
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

CombinedNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'SSCX_O1':
> Neuron IDs (10): [  5980   6148   7535 143800 144145 145307 146090 150847 159860 170760]
> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [5980, 6148, 7535, 143800, 144145, 145307, 146090, 150847, 159860, 170760]}


#### (e) __IDNeuronSet__, based on individual neuron IDs

In [8]:
neuron_set = obi.IDNeuronSet(neuron_ids=obi.NamedTuple(name="IDNeuronSet1", elements=(1, 2, 3)))
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

IDNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'SSCX_O1':
> Neuron IDs (3): [1 2 3]
> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [1, 2, 3]}


#### (f) __IDNeuronSet__, based on individual neuron IDs, with random sub-sampling
<u>Note</u>: `sample_percentage` can be an absolute number or fraction

In [ ]:
neuron_set = obi.IDNeuronSet(neuron_ids=obi.NamedTuple(name="IDNeuronSet1", elements=range(10)), sample_percentage=50, sample_seed=999)
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

IDNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'SSCX_O1':
> Neuron IDs (5): [1 4 5 6 8]
> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [1, 4, 5, 6, 8]}


#### (g) __PropertyNeuronSet__, based on neuron properties
<u>Note</u>: Optionally, instead of keeping the synbolic notation, neuron IDs can be resolved to individual IDs by `force_resolve_ids=True`.

In [10]:
neuron_set = obi.PropertyNeuronSet(
    property_filter=obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": ["2", "3"], "synapse_class": ["INH"]}),
)
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

# # Optional: Individual neuron IDs resolved
print(f"> Node set dict with IDs resolved [OPTIONAL]: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name, force_resolve_ids=True)}")

PropertyNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'SSCX_O1':
> Neuron IDs (7786): [ 23620  23621  23622 ... 170851 170852 170853]
> Node set dict: {'layer': ['2', '3'], 'synapse_class': 'INH'}
> Node set dict with IDs resolved [OPTIONAL]: {'population': 'S1nonbarrel_neurons', 'node_id': [23620, 23621, 23622, 23623, 23624, 23625, 23626, 23627, 23628, 23629, 23630, 23631, 23632, 23633, 23634, 23635, 23636, 23637, 23638, 23639, 23640, 23641, 23642, 23643, 23644, 23645, 23646, 23647, 23648, 23649, 23650, 23651, 23652, 23653, 23654, 23655, 23656, 23657, 23658, 23659, 23660, 23661, 23662, 23663, 23664, 23665, 23666, 23667, 23668, 23669, 23670, 23671, 23672, 23673, 23674, 23675, 23676, 23677, 23678, 23679, 23680, 23681, 23682, 23683, 23684, 23685, 23686, 23687, 23688, 23689, 23690, 23691, 23692, 23693, 23694, 23695, 23696, 23697, 23698, 23699, 23700, 23701, 23702, 23703, 23704, 23705, 23706, 23707, 23708, 23709, 23710, 23711, 23712, 23713, 23714, 23715, 23716, 23717, 2

#### (h) __PropertyNeuronSet__, based on neuron properties, combined with exising (named) node sets
<u>Note</u>: In this case, individual neuron IDs will always be resolved since a combination of properties and node sets is not possible in SONATA node sets otherwise!

In [11]:
neuron_set = obi.PropertyNeuronSet(
    property_filter=obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"synapse_class": ["INH"]}),
    node_sets=("Layer2", "Layer3")
)
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

PropertyNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'SSCX_O1':
> Neuron IDs (7786): [ 23620  23621  23622 ... 170851 170852 170853]
> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [23620, 23621, 23622, 23623, 23624, 23625, 23626, 23627, 23628, 23629, 23630, 23631, 23632, 23633, 23634, 23635, 23636, 23637, 23638, 23639, 23640, 23641, 23642, 23643, 23644, 23645, 23646, 23647, 23648, 23649, 23650, 23651, 23652, 23653, 23654, 23655, 23656, 23657, 23658, 23659, 23660, 23661, 23662, 23663, 23664, 23665, 23666, 23667, 23668, 23669, 23670, 23671, 23672, 23673, 23674, 23675, 23676, 23677, 23678, 23679, 23680, 23681, 23682, 23683, 23684, 23685, 23686, 23687, 23688, 23689, 23690, 23691, 23692, 23693, 23694, 23695, 23696, 23697, 23698, 23699, 23700, 23701, 23702, 23703, 23704, 23705, 23706, 23707, 23708, 23709, 23710, 23711, 23712, 23713, 23714, 23715, 23716, 23717, 23718, 23719, 23720, 23721, 23722, 23723, 23724, 23725, 23726, 23727, 23728, 23729, 23730, 23

#### (i) __VolumetricCountNeuronSet__, sample a spatial neighborhood


In [12]:
neuron_set = obi.VolumetricCountNeuronSet(
    ox=10.0,
    oy=25.0,
    oz=100.0,
    n=10,
    property_filter=obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": ["2", "3"], "synapse_class": ["INH"]}),
)

neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

print("Printing neuron locations to prove spatial compactness:")
circuit.sonata_circuit.nodes[circuit.default_population_name].get(neuron_ids, properties=["x", "y", "z"])

VolumetricCountNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'SSCX_O1':
> Neuron IDs (10): [148019 148107 148131 148447 148826 148892 149125 149968 151248 152115]
> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [148131, 148107, 149968, 149125, 151248, 148019, 148447, 152115, 148892, 148826]}
Printing neuron locations to prove spatial compactness:


,x,y,z
node_ids,,,
148019,4411.915879,-1389.507264,-1799.917421
148107,4433.904458,-1356.015771,-1810.152926
148131,4399.522448,-1347.995709,-1820.465085
148447,4365.399006,-1340.807643,-1786.213272
148826,4388.662824,-1313.595308,-1842.816309
148892,4356.211035,-1366.272885,-1825.001304
149125,4364.651608,-1355.809057,-1800.323261
149968,4381.265467,-1376.815146,-1831.129676
151248,4428.303479,-1346.771312,-1788.109583


#### (j) __VolmetricRadiusNeuronSet__, also neigborhood, but fixed radius
<u>Note</u>: Can be combined with randm subsampling

In [13]:
neuron_set = obi.VolumetricRadiusNeuronSet(
    ox=10.0,
    oy=25.0,
    oz=100.0,
    radius=75.0,
    property_filter=obi.scientific.circuit.neuron_sets.NeuronPropertyFilter(filter_dict={"layer": ["2", "3"], "synapse_class": ["INH"]}),
    sample_percentage=5
)

neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

print("Printing neuron locations to prove spatial compactness:")
circuit.sonata_circuit.nodes[circuit.default_population_name].get(neuron_ids, properties=["x", "y", "z"])

VolumetricRadiusNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'SSCX_O1':
> Neuron IDs (5): [147816 148022 148175 149125 150451]
> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [147816, 148022, 148175, 149125, 150451]}
Printing neuron locations to prove spatial compactness:


,x,y,z
node_ids,,,
147816,4444.843260,-1324.328559,-1863.332408
148022,4372.465956,-1365.394487,-1777.973673
148175,4443.588588,-1380.700301,-1835.718711
149125,4364.651608,-1355.809057,-1800.323261
150451,4434.830667,-1356.474542,-1783.596336


#### (k) __SimplexMembershipBasedNeuronSet__, choose neurons based on their membership in simplices with a given source or target neuronm, 
<u>Note</u>: Needs connalysis to run and needs a circuit with a connectivity matrix

In [2]:
from connalysis.network import topology # Neeeds to be installed to run this example

circuit_path_prefix = "/Users/danielaegas/OneDrive - Open Brain Institute/Shared Documents - OBI - Scientific staff/Data/Circuits hardcoded/"
circ_path = circuit_path_prefix + "Circuits_archived/MorphologyContainerizationTest/nbS1-HEX0-L23/circuit_config.json"
mat_path = circuit_path_prefix + "ConnectivityMatrices/nbS1-HEX0-L23/connectivity_matrix.h5"
circuit = obi.Circuit(name="nbS1-HEX0-L23", path=circ_path, matrix_path=mat_path)
print(f"Circuit '{circuit}' with {circuit.sonata_circuit.nodes.size} neurons and {circuit.sonata_circuit.edges.size} synapses")
print(f"Default node population: '{circuit.default_population_name}'")

Circuit 'nbS1-HEX0-L23' with 13016 neurons and 5822119 synapses
Default node population: 'S1nonbarrel_neurons'


In [3]:
neuron_set = obi.SimplexMembershipBasedNeuronSet(
    central_neuron_id = 4,#3115, 
    dim = 2,
    central_neuron_simplex_position = 'source',
    subsample = True ,
    n_count_max = 3,
    subsample_method = 'node_participation', #"random" is another option,
    property_filter=obi.scientific.circuit.neuron_sets.NeuronPropertyFilter( ),
)

neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"\n> Neuron IDs ({len(neuron_ids)}): {neuron_ids}\n")
print(f"\n> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

print("\n> Node selection\n")
sub_conn = circuit.connectivity_matrix.subpopulation(neuron_ids)
display(sub_conn.vertices)
print(f"> Simplex counts in subcircuit:\n {topology.simplex_counts(sub_conn.matrix)}\n")




[2025-06-18 15:01:26,756] INFO: COMPUTE list of simplices by dimension
> n_count_max is too small to form a single 2-simplex, sampling n_count_max = 3 neurons instead.
SimplexMembershipBasedNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'nbS1-HEX0-L23':

> Neuron IDs (3): [   4   16 3451]

[2025-06-18 15:01:27,317] INFO: COMPUTE list of simplices by dimension
> n_count_max is too small to form a single 2-simplex, sampling n_count_max = 3 neurons instead.

> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [4, 3451, 16]}

> Node selection



,node_ids,etype,layer,mtype,synapse_class,x,y,z
0,4,cADpyr,3,L3_TPC:A,EXC,4375.797338,-1368.977633,-2091.973131
1,16,cADpyr,3,L3_TPC:A,EXC,4365.489707,-1344.641437,-2110.681247
2,3451,cACint,3,L23_CHC,INH,4426.676668,-1365.720303,-1999.555547


> Simplex counts in subcircuit:
 dim
0    3
1    3
2    1
Name: simplex_count, dtype: int64



#### (l) __SimplexMembershipBasedNeuronSet__, choose neurons based on their membership in simplices with a given source or target neuronm, 
<u>Note</u>: Needs connalysis to run and needs a circuit with a connectivity matrix

In [4]:
neuron_set = obi.SimplexNeuronSet(
    central_neuron_id = 4, 
    dim = 100,
    central_neuron_simplex_position = 'source',
    subsample = True ,
    n_count_max = 3,
    property_filter=obi.scientific.circuit.neuron_sets.NeuronPropertyFilter( filter_dict={"synapse_class": ["EXC"]}),
)

neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"\n> Neuron IDs ({len(neuron_ids)}): {neuron_ids}\n")
print(f"\n> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

print("\n> Node selection\n")
sub_conn = circuit.connectivity_matrix.subpopulation(neuron_ids)
display(sub_conn.vertices)
print(f"> Simplex counts in subcircuit:\n {topology.simplex_counts(sub_conn.matrix)}\n")


[2025-06-18 15:01:32,674] INFO: COMPUTE list of simplices by dimension
> Dimension not attained using dimension 4 instead.
> n_count_max is too small to form a single 4-simplex, sampling n_count_max = 5 neurons instead.
SimplexNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'nbS1-HEX0-L23':

> Neuron IDs (5): [   4 1710 2443 6522 7911]

[2025-06-18 15:01:33,155] INFO: COMPUTE list of simplices by dimension
> Dimension not attained using dimension 4 instead.
> n_count_max is too small to form a single 4-simplex, sampling n_count_max = 5 neurons instead.

> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [4, 1710, 2443, 6522, 7911]}

> Node selection



,node_ids,etype,layer,mtype,synapse_class,x,y,z
0,4,cADpyr,3,L3_TPC:A,EXC,4375.797338,-1368.977633,-2091.973131
1,1710,cADpyr,3,L3_TPC:A,EXC,4397.464066,-1330.506049,-2071.480830
2,2443,cADpyr,3,L3_TPC:A,EXC,4504.030195,-1300.462459,-2091.379527
3,6522,cADpyr,2,L2_TPC:B,EXC,4557.486658,-1300.461762,-1948.439484
4,7911,cADpyr,2,L2_TPC:B,EXC,4585.857757,-1315.662188,-1951.827403


> Simplex counts in subcircuit:
 dim
0     5
1    10
2    10
3     5
4     1
Name: simplex_count, dtype: int64



### __Example 3:__ Writing a NeuronSet to a SONATA node set file
<u>Note</u>: A NeuronSet name must be set, which will be the name of the SONATA node set. The name must not exist!

<u>Note 2</u>: The node sets file name is by default taken from the original circuit. An alternative name can optionally be provided.

<u>Note 3</u>: Overwrite (`overwrite_if_exists`) and append (`append_if_exists`) options exist.

In [14]:
output_path = "./"

# Write new file, overwrite if existing
neuron_set = obi.CombinedNeuronSet(node_sets=("Layer1", "Layer2", "Layer3"))
nset_file = neuron_set.to_node_set_file(circuit, circuit.default_population_name, output_path=output_path, overwrite_if_exists=True, optional_node_set_name="L123")

# Append to existing file, but name already exists => NOT POSSIBLE
# nset_file = neuron_set.to_node_set_file(circuit, circuit.default_population_name, output_path="./", append_if_exists=True)  # AssertionError: Appending not possible, node set 'Basic' already exists!

# Append to existing file
neuron_set = obi.CombinedNeuronSet(node_sets=("Layer4", "Layer5", "Layer6"))
nset_file = neuron_set.to_node_set_file(circuit, circuit.default_population_name, output_path=output_path, append_if_exists=True, optional_node_set_name="L456")

if os.path.exists(nset_file):
    print(f"Node set file: {nset_file}")

Node set file: ./node_sets.json
